# Aspect Based Sentiment Analysis


### Data And Code

In [2]:
# import all necessary packeges
import pandas as pd
import xml.etree.ElementTree as ET, getopt, logging, sys, random, re, copy
from xml.sax.saxutils import escape
import io


from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np

import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")







## Fetching Data

In [24]:
# we have give a json file which consists of all information about
# the products
# loading the data using pandas' read_json file.
xml_data1 = open('D:\ABSA\semeval_data\Restaurants_Train.xml').read()

# Fetch the sentence data
def xml2df(xml_data):
    root = ET.XML(xml_data) # element tree
    all_records = [] #This is our record list which we will convert into a dataframe
    for i, child in enumerate(root): #Begin looping through our root tree
        record = {} #Place holder for our record
        for subchild in child: #iterate through the subchildren to user-agent, Ex: ID, String, Description.
            if(subchild.tag == 'text'):
                record[subchild.tag] = subchild.text
                all_records.append(record)
            #record[subchild.tag] = subchild.text #Extract the text create a new dictionary key, value pair
            #print(subchild.text)
            #all_records.append(record) #Append this record to all_records.
            
    return pd.DataFrame(all_records) #return records as DataFrame


def iter_docs(sentences):
    author_attr = sentences.attrib
    for doc in sentences.iter('sentence'):
        
        #print(doc.find('text'))
        doc_dict = author_attr.copy()
        #print(author_attr['aspectTerm'])
        #print(doc.attrib)
        doc_dict.update(doc.attrib)  # doc.attribute give the value of the sentence id e.g. {'id': 12334}
        #doc_dict['data'] = doc.text
        #print(doc_dict)
        yield doc_dict
        


xml_data = io.StringIO(xml_data1)

etree = ET.parse(xml_data) #create an ElementTree object 
doc_df = pd.DataFrame(list(iter_docs(etree.getroot())))


sentence_data = xml2df(xml_data1)



print(doc_df)
print(sentence_data)

        id
0     3121
1     2777
2     1634
3     2534
4      583
5     2846
6     1571
7     1458
8     3161
9     2391
10     986
11     296
12    1793
13    2200
14    3195
15    3119
16    3359
17      70
18    1829
19    2935
20    1805
21    2559
22     716
23    2464
24     385
25    1350
26    3534
27    1407
28     565
29     387
...    ...
3014   902
3015   568
3016  3133
3017  2553
3018  2624
3019  2116
3020   317
3021  2151
3022   944
3023  3014
3024   900
3025  2739
3026   279
3027  2937
3028   333
3029   236
3030  1420
3031  2259
3032   608
3033  2834
3034  3168
3035  2798
3036  2378
3037  1027
3038  1735
3039  1063
3040   777
3041   875
3042   671
3043   617

[3044 rows x 1 columns]
                                                   text
0                  But the staff was so horrible to us.
1     To be completely fair, the only redeeming fact...
2     The food is uniformly exceptional, with a very...
3     Where Gabriela personaly greets you and recomm...
4     For tho

In [102]:
print(sum(sentence_data.duplicated('text')))

6


#### For the Aspect Term Data

In [25]:
aspects_records = []
for node in etree.iter():
    record = {}
    
    #print(node.attrib)
    #print(node.tag)
    
    
    if(node.tag == 'sentence'):
        id = node.attrib.get('id')
        #print("1:",id)
        
    if(node.tag == 'aspectTerm' ):
        
        #print("2",id)
        record['id'] = id
        record['term'] = node.attrib.get('term')
        record['polarity'] = node.attrib.get('polarity')
        record['from'] = node.attrib.get('from')
        record['to'] = node.attrib.get('to')
        aspects_records.append(record)
    #print(node.attrib.get('id'))

    
    #record[node.tag] = node.attrib
    #all_records.append(record)
    
    #pd.DataFrame(all_records)
    #print ("Tag:",node.tag)
    
    #print("Sentence:",node.find('text').text)
    
    #print("Value:", node.attrib.get('id'))
    
    
    #print (node.attrib) record['aaa'] = '111'
   # print ("Term:",node.attrib.get('term'),"Polarity:", node.attrib.get('polarity'), "From:", node.attrib.get('From'), "To", node.attrib.get('to'))
   # print ("Polarity", node.attrib.get('polarity'))
   #print ("From", node.attrib.get('to'))



#print(doc_df1)

In [8]:

aspect_terms = pd.DataFrame(aspects_records)
aspect_terms

,from,id,polarity,term,to
0,8,3121,negative,staff,13
1,57,2777,positive,food,61
2,4,1634,positive,food,8
3,55,1634,positive,kitchen,62
4,141,1634,neutral,menu,145
5,17,2846,positive,food,21
6,51,2846,positive,perks,56
7,27,1458,positive,orrechiete with sausage and chicken,62
8,76,1458,positive,waiters,83
9,152,1458,neutral,meats,157


### Sentence Catergory Table

In [26]:
aspect_category_records = []
for node in etree.iter():
    record = {} 
    #print(node.attrib)
    #print(node.tag)
    
    
    if(node.tag == 'sentence'):
        id = node.attrib.get('id')
        #print("1:",id)
        
    if(node.tag == 'aspectCategory' ):
        
        #print("2",id)
        record['id'] = id
        record['category'] = node.attrib.get('category')
        record['polarity'] = node.attrib.get('polarity')
        
        aspect_category_records.append(record)

In [27]:
aspect_category = pd.DataFrame(aspect_category_records)
aspect_category

,category,id,polarity
0,service,3121,negative
1,food,2777,positive
2,anecdotes/miscellaneous,2777,negative
3,food,1634,positive
4,service,2534,positive
5,anecdotes/miscellaneous,583,positive
6,food,2846,positive
7,service,2846,positive
8,food,1571,negative
9,price,1571,negative


In [12]:
print("Number of data points", doc_df.shape[0])
print("Number of data points", doc_df.shape[1])
print("Number of data points", sentence_data.shape[0])
print("Number of data points", sentence_data.shape[1])

Number of data points 3044
Number of data points 1
Number of data points 3044
Number of data points 1


In [29]:
text_data = pd.concat([doc_df, sentence_data], axis=1)

In [30]:
text_data

,id,text
0,3121,But the staff was so horrible to us.
1,2777,"To be completely fair, the only redeeming fact..."
2,1634,"The food is uniformly exceptional, with a very..."
3,2534,Where Gabriela personaly greets you and recomm...
4,583,"For those that go once and don't enjoy it, all..."
5,2846,"Not only was the food outstanding, but the lit..."
6,1571,It is very overpriced and not very tasty.
7,1458,Our agreed favorite is the orrechiete with sau...
8,3161,The Bagels have an outstanding taste with a te...
9,2391,Nevertheless the food itself is pretty good.


In [31]:
text_data.columns

Index(['id', 'text'], dtype='object')

In [83]:
print("Number of features points", doc_df.shape[1])

Number of features points 1


In [84]:
doc_df.columns

Index(['id'], dtype='object')

### Basic Status of the data
1. Sentence Table (text_data)
2. Aspect Terms Data (aspect_terms)
3. Category Data (aspect_category)

In [55]:
#colums in text_data
text_data.columns

Index(['id', 'text'], dtype='object')

In [56]:
#describe text data
print(text_data['text'].describe())

count           3044
unique          3038
top       Good food.
freq               2
Name: text, dtype: object


In [88]:
#Aspect colums 
aspect_terms.columns

Index(['from', 'id', 'polarity', 'term', 'to'], dtype='object')

In [89]:
#Describe term in the aspects
print(aspect_terms['term'].describe())

count     3699
unique    1295
top       food
freq       358
Name: term, dtype: object


In [90]:
# Category columns
aspect_category.columns

Index(['category', 'id', 'polarity'], dtype='object')

In [91]:
# Describe category 
print(aspect_category['category'].describe())

count     3714
unique       5
top       food
freq      1233
Name: category, dtype: object


In [93]:
aspect_terms['term'].unique()

array(['staff', 'food', 'kitchen', ..., 'cheese sticks',
       'pot of boiling water', 'glass noodles'], dtype=object)

In [94]:
aspect_category['category'].unique()

array(['service', 'food', 'anecdotes/miscellaneous', 'price', 'ambience'], dtype=object)

In [105]:
# store thr sentence text data into the pickels/180k_text_data
text_data.to_pickle('pickels/180k_text_data')
aspect_terms.to_pickle('pickels/180k_aspect_data')
aspect_category.to_pickle('pickels/180k_category_data')

In [106]:
print("thie sedif dfdf dfdf".split())

['thie', 'sedif', 'dfdf', 'dfdf']


## 2. Text Preprocessing

In [336]:
# Remove the special charecter in the string 

string = "special character $# df_*d dfa chara spaces 88823434"
''.join(e for e in string if e.isalnum())


'specialcharacterdfddfacharaspaces88823434'

In [38]:
# I use the the  list of stop words that are downloaded from nltk lib.

stop_words = set(stopwords.words('english'))
print ('list of stop words:', stop_words)


def nlp_stopwordRemove(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            word = word.lower()
            # Remove stop words
            if not word in stop_words:
                string += word + " "
    text_data[column][index] = string

list of stop words: {'at', 'down', 's', 'very', 'once', 'yourselves', 'with', 'being', 'himself', 'as', 'them', 'is', 'theirs', 'haven', 'wasn', 'shouldn', 'through', 'am', 'weren', 'or', 'shan', 'aren', 'hers', 'in', 'ain', 'while', 'o', 'wouldn', 'did', 'our', 'having', 'above', 'myself', 'and', 'under', 'few', 'they', 'have', 'other', 've', 'into', 'own', 'yourself', 'mightn', 'him', 'ours', 'some', 'his', 'm', 're', 'my', 'off', 'be', 'its', 'between', 'there', 'same', 'been', 'until', 'on', 'such', 'an', 'how', 'nor', 'themselves', 'itself', 'any', 'here', 'but', 'doesn', 'again', 'y', 'does', 'by', 'herself', 'her', 'out', 'it', 'has', 'all', 'their', 'if', 'this', 'whom', 'where', 'each', 'further', 'a', 'mustn', 'only', 'should', 'will', 'for', 'up', 'from', 'doing', 'over', 'can', 'both', 'what', 'that', 'no', 'when', 'about', 'against', 'he', 'i', 'of', 'isn', 'now', 'll', 'who', 'then', 'ma', 'won', 'was', 'me', 'ourselves', 't', 'just', 'couldn', 'didn', 'these', 'do', 'too

In [39]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    nlp_stopwordRemove(row['text'], index, 'text')
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

0.6343066110141535 seconds


In [21]:
text_data

,id,text
0,3121,staff horrible us
1,2777,completely fair redeeming factor food average ...
2,1634,food uniformly exceptional capable kitchen pro...
3,2534,gabriela personaly greets recommends eat
4,583,go dont enjoy say dont get
5,2846,food outstanding little perks great
6,1571,overpriced tasty
7,1458,agreed favorite orrechiete sausage chicken usu...
8,3161,bagels outstanding taste terrific texture chew...
9,2391,nevertheless food pretty good


In [59]:
print(text_data['text'].describe())

count                    3044
unique                   3008
top       highly recommended 
freq                        4
Name: text, dtype: object


In [60]:
product_type_count = Counter(list(text_data['text']))
#product_type_count
product_type_count.most_common(40)

[('food great ', 4),
 ('highly recommended ', 4),
 ('love place ', 4),
 ('back ', 3),
 ('', 3),
 ('go ', 3),
 ('wonderful ', 3),
 ('disappointed ', 2),
 ('cant wait go back ', 2),
 ('check ', 2),
 ('dont go ', 2),
 ('enjoy ', 2),
 ('pleasantly surprised ', 2),
 ('finally ', 2),
 ('would highly recommend ', 2),
 ('great place ', 2),
 ('good food ', 2),
 ('horrible ', 2),
 ('delicious ', 2),
 ('really ', 2),
 ('amazing ', 2),
 ('definitely go back ', 2),
 ('food good ', 2),
 ('loved ', 2),
 ('unbelievable ', 2),
 ('service excellent food delicious ', 2),
 ('staff horrible us ', 1),
 ('completely fair redeeming factor food average couldnt make deficiencies teodora ',
  1),
 ('food uniformly exceptional capable kitchen proudly whip whatever feel like eating whether menu ',
  1),
 ('gabriela personaly greets recommends eat ', 1),
 ('go dont enjoy say dont get ', 1),
 ('food outstanding little perks great ', 1),
 ('overpriced tasty ', 1),
 ('agreed favorite orrechiete sausage chicken usually

In [61]:
data11 = ""
#for row in text_data.iterrows():
    #print(row['text'])
    #data11 += row['text']
sent = ""
for index, row in text_data.iterrows():
    sent += row['text']    
    
#print(sent.split(" "))

print(len(sent))

less_freq = {}


cou = Counter(list(sent.split(" ")))
print(cou)
for k,v in less_freq:
    if(v == 1):
        less_freq.add(k)
        print(k)

print(less_freq)
        

#print(len(cou))

less_freq = cou.most_common()[:-2435-1:-1]




#print (cou.values )

#for v in cou:


#print (data11.split())

'''
print(data11.split())
product_type_count = Counter(list(text_data['text']))
#product_type_count
product_type_count.most_common(40)
#.most_common(40)

#data1111 = data11.split()
'''





143497
Counter({'food': 453, 'place': 295, 'great': 261, 'good': 261, 'service': 247, 'restaurant': 189, 'go': 166, '': 165, 'like': 132, 'one': 123, 'best': 115, 'get': 110, 'would': 107, 'back': 92, 'dont': 88, 'ive': 85, 'time': 85, 'always': 85, 'delicious': 84, 'pizza': 83, 'menu': 82, 'try': 79, 'even': 77, 'dinner': 76, 'never': 75, 'wine': 74, 'nice': 74, 'well': 74, 'really': 72, 'staff': 71, 'excellent': 71, 'us': 70, 'table': 67, 'times': 66, 'people': 66, 'prices': 66, 'wait': 65, 'recommend': 64, 'little': 63, 'night': 63, 'friendly': 61, 'also': 61, 'better': 61, 'eat': 60, 'went': 59, 'thai': 59, 'experience': 58, 'restaurants': 57, 'sushi': 56, 'ever': 53, 'atmosphere': 52, 'new': 51, 'worth': 50, 'fresh': 50, 'dishes': 49, 'friends': 49, 'small': 49, 'price': 48, 'though': 48, 'could': 46, 'love': 46, 'special': 45, 'indian': 45, 'meal': 44, 'first': 44, 'nyc': 44, 'bar': 43, 'lunch': 43, 'chicken': 42, 'definitely': 42, 'got': 42, 'make': 41, 'city': 41, 'everything':

"\nprint(data11.split())\nproduct_type_count = Counter(list(text_data['text']))\n#product_type_count\nproduct_type_count.most_common(40)\n#.most_common(40)\n\n#data1111 = data11.split()\n"

### The seed words of each category
##### Unique Categories are 'service', 'food', 'anecdotes/miscellaneous', 'price', 'ambience'.

In [62]:
from nltk.corpus import wordnet
service_syns = wordnet.synsets("service")
print(service_syns[0].definition())
print(service_syns[0].examples())

#print (len(service_syns))
#i = len(service_syns)
for i in range(len(service_syns)):
    print(service_syns[i].lemmas()[0].name())

food_syns = wordnet.synsets("food")
print (food_syns)
for i in range(len(food_syns)):
    print(food_syns[i].lemmas()[0].name())

anecdotes_syns = wordnet.synsets("anecdotes")
print(anecdotes_syns)
anecdotes_syns = wordnet.synsets("miscellaneous")
print(anecdotes_syns)

pri = wordnet.synsets("price")
print(pri)

amb = wordnet.synsets("ambience")
print(amb)
#syns[0].name()
#syns[0].lemmas()[0].name()

work done by one person or group that benefits another
['budget separately for goods and services']
service
service
service
service
service
military_service
Service
avail
service
servicing
service
serve
service
overhaul
service
service
service
serve
[Synset('food.n.01'), Synset('food.n.02'), Synset('food.n.03')]
food
food
food
[Synset('anecdote.n.01')]
[Synset('assorted.s.01'), Synset('many-sided.s.02')]
[Synset('monetary_value.n.01'), Synset('price.n.02'), Synset('price.n.03'), Synset('price.n.04'), Synset('price.n.05'), Synset('price.n.06'), Synset('price.n.07'), Synset('price.v.01'), Synset('price.v.02')]
[Synset('atmosphere.n.01'), Synset('ambiance.n.02')]


In [240]:
import pandas as pd

df = pd.DataFrame({'TFD' : ['AA', 'SL', 'BB', 'D0', 'Dk', 'FF'],
                    'Snack' : ['1', '0', '1', '1', '0', '0'],
                    'Trans' : ['1', '1', '1', '0', '0', '1'],
                    'Dop' : ['1', '0', '1', '0', '1', '1']}).set_index('TFD')

print(df)

    Dop Snack Trans
TFD                
AA    1     1     1
SL    0     0     1
BB    1     1     1
D0    0     1     0
Dk    1     0     0
FF    1     0     1


In [241]:
df_asint = df.astype(int)
coocc = df_asint.T.dot(df_asint)
coocc

,Dop,Snack,Trans
Dop,4,2,3
Snack,2,3,2
Trans,3,2,4


In [242]:
np.fill_diagonal(coocc.values, 0)

In [243]:
coocc

,Dop,Snack,Trans
Dop,0,2,3
Snack,2,0,2
Trans,3,2,0


In [244]:
import scipy.sparse as sp
X = sp.csr_matrix(df.astype(int).values) # convert dataframe to sparse matrix
Xc = X.T * X # multiply sparse matrix # 
Xc.setdiag(0) # reset diagonal
print(Xc.todense()) # to print co-occurence matrix in dense format


[[0 2 3]
 [2 0 2]
 [3 2 0]]


In [355]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("texture"))

texture


##    Lemmatization

In [33]:
lemmatizer = WordNetLemmatizer()
def nlp_lemmatizing(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            word = word.lower()
            word = lemmatizer.lemmatize(word)
            string += word + " "
    text_data[column][index] = string


In [34]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    nlp_lemmatizing(row['text'], index, 'text')

# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")


3.0715904010304307 seconds


In [36]:
text_data

,id,text
0,3121,but the staff wa so horrible to u
1,2777,to be completely fair the only redeeming facto...
2,1634,the food is uniformly exceptional with a very ...
3,2534,where gabriela personaly greets you and recomm...
4,583,for those that go once and dont enjoy it all i...
5,2846,not only wa the food outstanding but the littl...
6,1571,it is very overpriced and not very tasty
7,1458,our agreed favorite is the orrechiete with sau...
8,3161,the bagel have an outstanding taste with a ter...
9,2391,nevertheless the food itself is pretty good


In [72]:
product_type_count = Counter(list(text_data['text']))
product_type_count

Counter({'staff wa horrible u ': 1,
         'completely fair wa food wa average couldnt make ': 1,
         'food exceptional kitchen feel like eating whether menu ': 1,
         'eat ': 1,
         'go dont enjoy say dont get ': 1,
         'wa food outstanding little great ': 1,
         'overpriced tasty ': 1,
         'favorite chicken usually waiter kind enough split dish half get meat ': 1,
         'bagel outstanding taste terrific yet ': 1,
         'food pretty good ': 1,
         'ive noticed st traditional vietnamese black ': 1,
         'forgot toast left ingredient cheese hot wa cooked plate ': 1,
         'took half hour get check wa perfect since could sit drink talk ': 1,
         'atmosphere good ': 1,
         'ha visited quite cuisine ': 1,
         'would definitely recommend mary making one regular neighborhood ': 1,
         'pizza best like thin pizza ': 1,
         'girlfriend stumbled place night great time ': 1,
         'cant help theyre bought fast ': 1,
  

In [79]:
from itertools import dropwhile
sent = ""
for index, row in text_data.iterrows():
    sent += row['text']
#print (sent)
    
#print(sent.split(" "))

print(len(sent))

less_freq = {}

cou = Counter(list(sent.split(" ")))



cou1 = cou

#cou
#print(cou)

print (type(cou))

lis = list(cou1.items())
lis
#print(type(lis))
lis= set(lis)
#print (lis)
#print(type(lis))


for key, count in dropwhile(lambda key_count: key_count[1] > 2, cou.most_common()):
     del cou[key]
print (cou)



print (len(cou))



116480
<class 'collections.Counter'>
Counter({'wa': 626, 'food': 464, 'place': 330, 'great': 261, 'good': 261, 'service': 247, 'restaurant': 246, 'go': 170, 'time': 151, 'like': 134, 'one': 126, 'get': 119, 'best': 115, 'price': 114, 'would': 107, 'back': 92, 'pizza': 89, 'dont': 88, 'ha': 87, 'ive': 85, 'always': 85, 'delicious': 84, 'menu': 83, 'table': 83, 'wine': 82, 'dish': 81, 'try': 81, 'dinner': 78, 'even': 77, 'friend': 75, 'never': 75, 'nice': 74, 'well': 74, 'staff': 72, 'u': 72, 'really': 72, 'night': 71, 'excellent': 71, 'people': 66, 'wait': 65, 'recommend': 64, 'little': 63, 'special': 63, 'experience': 62, 'friendly': 61, 'also': 61, 'better': 61, 'make': 60, 'eat': 60, 'went': 59, 'thai': 59, 'sushi': 56, 'drink': 55, 'ever': 53, 'atmosphere': 52, 'new': 51, 'worth': 50, 'fresh': 50, 'small': 49, 'love': 49, 'though': 48, 'meal': 47, 'could': 46, 'indian': 45, 'bar': 44, 'first': 44, 'nyc': 44, 'order': 44, 'say': 43, 'come': 43, 'lunch': 43, 'chicken': 42, 'definitely

In [74]:
text_data

,id,text
0,3121,staff wa horrible u
1,2777,completely fair wa food wa average couldnt make
2,1634,food exceptional kitchen feel like eating whet...
3,2534,eat
4,583,go dont enjoy say dont get
5,2846,wa food outstanding little great
6,1571,overpriced tasty
7,1458,favorite chicken usually waiter kind enough sp...
8,3161,bagel outstanding taste terrific yet
9,2391,food pretty good


##### Total Number of words remain is 4113
# of single word 2199 (4113 - 2199 = 1914)
# of single and two count words 2818 remaning(4113 - 2818 = 1295)


In [67]:
# Create a list whose frequency is greater than 2

#print(cou)

#cou = Counter(list(cou.split(" ")))
#print(cou)
#print (cou)




low_frequency_word = Counter(list(sent.split(" ")))

low_fre_word = []




cnt = 0

#print (low_frequency_word)


for k, v in low_frequency_word.items():
    if(v < 3):
        cnt = cnt + 1
        low_fre_word.append(k)
        
#print(low_frequency_word)
   
print (low_fre_word) 
        
#print (cnt)


listt = Counter(low_fre_word)


#print (listt)
listt = set(listt)

#print (listt)

#print (len(myList))
#print(low_fre_word)




lis2= set(low_fre_word)

#print (low_fre_word)
#print (lis2)
#print(type(lis2))



#print (lis - listt)





staff
72
wa
626
horrible
13
u
72
completely
8
fair
5
redeeming
1
factor
2
food
464
average
25
couldnt
10
make
60
deficiency
1
teodora
1
uniformly
1
exceptional
4
capable
2
kitchen
10
proudly
1
whip
1
whatever
2
feel
24
like
134
eating
21
whether
6
menu
83
gabriela
1
personaly
1
greets
1
recommends
1
eat
60
go
170
dont
88
enjoy
20
say
43
get
119
outstanding
14
little
63
perk
1
great
261
overpriced
13
tasty
37
agreed
1
favorite
33
orrechiete
1
sausage
2
chicken
42
usually
19
waiter
40
kind
18
enough
24
split
5
dish
81
half
12
sample
2
meat
10
bagel
39
taste
36
terrific
13
texture
2
chewy
2
yet
22
gummy
1
nevertheless
2
pretty
27
good
261
ive
85
noticed
5
megadeth
1
mr
1
scruff
1
st
12
germain
1
traditional
7
vietnamese
5
song
1
black
8
sabbath
1
jayz
1
daft
1
punk
1
played
1
mayonnaise
1
forgot
5
toast
4
left
21
ingredient
12
ie
2
cheese
31
omelet
1
hot
32
temperature
1
bacon
2
cooked
13
crumbled
1
plate
18
touched
2
took
20
hour
19
check
20
perfect
29
since
22
could
46
sit
20
drink
55
t

7
kid
8
scared
1
entertained
1
sooo
2
skimp
1
outdoor
8
yorkie
1
mesclun
1
twenty
4
curtain
1
call
13
24hrs
1
knowledgable
2
chilaquiles
1
quibble
1
offset
1
pouryourown
1
measure
1
liquers
1
courtesey
1
industry
1
understood
1
mindblowing
1
budget
1
belly
1
longer
4
fat
2
meltinyourmouth
1
cheesy
2
yell
1
kinda
3
nosiy
1
nicky
1
treat
8
ruby
3
foos
3
havent
3
kicked
1
thatthe
1
dressed
4
lemon
4
virgnin
1
olive
4
oil
8
lexington
1
dainty
1
buttah
1
icy
1
wash
1
mushroomthird
1
generationfornini
1
drenched
1
truffle
2
nothingon
1
omelletes
1
sala
1
wellportioned
1
men
1
theyd
4
whenever
2
fix
5
sigh
1
ant
1
website
4
rating
4
reality
3
perfection
4
art
1
sidewalk
1
50
4
topping
7
event
3
brick
2
gallery
2
add
6
aunthentic
1
growing
5
start
7
hearty
3
thumb
2
scallion
2
pancake
3
watering
2
patron
3
undercooked
3
charged
2
45
4
notch
2
inconsistent
1
dumplingetc
1
guy
10
rock
3
fondue
4
careful
3
request
5
octopus
2
eater
3
floored
1
using
2
msg
1
wed
3
tom
2
kha
1
bombay
3
vibrant
1
ey

1
prefer
2
quiter
1
diverse
4
chill
1
flaky
1
apple
2
tarte
1
tatin
1
step
1
cosette
1
miraculously
1
offthebeaten
1
path
2
parisian
2
smitten
1
195
1
400
1
actor
3
definintely
1
urged
1
miso
1
preview
1
appreciation
1
redone
1
itsuperb
1
rao
1
incompetency
1
quail
1
negative
1
concerning
1
mashed
2
potato
6
blend
2
butter
3
fails
1
draw
1
definetly
2
worker
3
female
1
skipped
1
mixed
2
snack
1
mud
1
warmed
1
scoop
2
dulce
1
de
1
leche
1
stauff
1
furniture
1
tiffin
1
overhauled
1
cole
1
knish
2
530
1
wood
1
premium
1
appetite
2
aesthetic
2
beautification
1
sheswaycuterthanmethatb
1
tikkamasala
1
cornelia
1
broadway
1
po
2
require
1
skiline
1
foggy
1
rainy
1
seew
1
cashew
1
nut
1
memorable
4
repeatable
1
extraordinary
2
lively
1
sligtly
1
thius
1
appear
1
premisis
1
dirty
1
unkempt
1
warning
2
attract
1
crackling
1
disaster
1
shockingly
1
contract
1
deposit
1
mezz
1
grp
1
tea
6
downside
2
nazilike
1
complains
1
weary
2
untasty
1
flair
1
importantly
1
buon
1
appetito
1
roofdeck
1
somethi

In [68]:

# I use the the  list of stop words that have frequency 1.



def remove_lowFrequencyWord(total_text, index, column):
    #print(total_text)
    if type(total_text) is not int:
        string = ""
        for words in total_text.split():
            word = ''.join(e for e in words if e.isalnum())
            # Remove stop words
            if not word in low_fre_word:
                string += word + " "
    text_data[column][index] = string



In [75]:
start_time = time.clock()
# we take each title and we text-preprocess it.
for index, row in text_data.iterrows():
    remove_lowFrequencyWord(row['text'], index, 'text')
# we print the time it took to preprocess whole titles 
print(time.clock() - start_time, "seconds")

1.4252651929909916 seconds


In [89]:
# After removing low frequency words text_data
text_data

,id,text
0,3121,staff wa horrible u
1,2777,completely fair wa food wa average couldnt make
2,1634,food exceptional kitchen feel like eating whet...
3,2534,eat
4,583,go dont enjoy say dont get
5,2846,wa food outstanding little great
6,1571,overpriced tasty
7,1458,favorite chicken usually waiter kind enough sp...
8,3161,bagel outstanding taste terrific yet
9,2391,food pretty good


In [91]:
print (cou)

Counter({'wa': 626, 'food': 464, 'place': 330, 'great': 261, 'good': 261, 'service': 247, 'restaurant': 246, 'go': 170, 'time': 151, 'like': 134, 'one': 126, 'get': 119, 'best': 115, 'price': 114, 'would': 107, 'back': 92, 'pizza': 89, 'dont': 88, 'ha': 87, 'ive': 85, 'always': 85, 'delicious': 84, 'menu': 83, 'table': 83, 'wine': 82, 'dish': 81, 'try': 81, 'dinner': 78, 'even': 77, 'friend': 75, 'never': 75, 'nice': 74, 'well': 74, 'staff': 72, 'u': 72, 'really': 72, 'night': 71, 'excellent': 71, 'people': 66, 'wait': 65, 'recommend': 64, 'little': 63, 'special': 63, 'experience': 62, 'friendly': 61, 'also': 61, 'better': 61, 'make': 60, 'eat': 60, 'went': 59, 'thai': 59, 'sushi': 56, 'drink': 55, 'ever': 53, 'atmosphere': 52, 'new': 51, 'worth': 50, 'fresh': 50, 'small': 49, 'love': 49, 'though': 48, 'meal': 47, 'could': 46, 'indian': 45, 'bar': 44, 'first': 44, 'nyc': 44, 'order': 44, 'say': 43, 'come': 43, 'lunch': 43, 'chicken': 42, 'definitely': 42, 'got': 42, 'city': 41, 'everyt

In [120]:
# storing Data after revoming stop words and low frequency words in pickel file



couter_list = list(cou)
#couter_list_df = pd.DataFrame(couter_list)
couter_list_df = pd.DataFrame.from_dict(cou, orient='index').reset_index()

text_data.to_pickle('pickels/180k_preprocessed_data')



couter_list_df.to_pickle('pickels/80k_couter_data')


In [121]:
couter_list_df

,index,0
0,staff,72
1,wa,626
2,horrible,13
3,u,72
4,completely,8
5,fair,5
6,food,464
7,average,25
8,couldnt,10
9,make,60


##### Download Data form the Pickel






In [3]:
text_data = pd.read_pickle('pickels/180k_preprocessed_data')
counter_data = pd.read_pickle('pickels/80k_couter_data')

In [5]:
counter_data

,index,0
0,staff,72
1,wa,626
2,horrible,13
3,u,72
4,completely,8
5,fair,5
6,food,464
7,average,25
8,couldnt,10
9,make,60


In [7]:
counter_data.columns = ['Word', 'Frequency']

In [8]:
counter_data

,Word,Frequency
0,staff,72
1,wa,626
2,horrible,13
3,u,72
4,completely,8
5,fair,5
6,food,464
7,average,25
8,couldnt,10
9,make,60


##### Creating the Zeors couter Data lenght

In [134]:


A = np.random.randint(0, 10, size=36).reshape(6, 6)
names = [_ for _ in 'abcdef']
df = pd.DataFrame(A, index=names, columns=names)
#df.to_csv('df.csv', index=True, header=True, sep=' ')

print (df)
#df.iloc['a']['a']
print (df.loc['a','b'])



# print (words_data_list)






   a  b  c  d  e  f
a  4  2  8  3  4  1
b  0  3  7  6  3  7
c  5  8  3  2  8  7
d  7  0  2  8  7  1
e  7  4  3  8  4  7
f  6  0  7  5  4  8
2
<class 'list'>


In [ ]:
X = Zeros

In [11]:
counter_data.columns

words_data_list = counter_data['Word'].tolist()
X = np.zeros((len(counter_data), len(counter_data)))
names = [_ for _ in words_data_list ]
df = pd.DataFrame(X, index=names, columns=names)

df

,staff,wa,horrible,u,completely,fair,food,average,couldnt,make,...,upstairs,tofu,gnochi,superb,delightfully,etc,frequently,showed,hidden,nosh
staff,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
wa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
horrible,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
u,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
completely,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
fair,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
average,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
couldnt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
make,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Sentence in the file
text_data['text']

#print (text_data.iloc[0]['text'])

#word = "staff"

for i, word in enumerate (counter_data['Word']):
    for i,line in enumerate(text_data['text']):
        if word in line: # or word in line.split() to search for full words
            print("Word \"{}\" found in line {}".format(word, i))
            mystring = text_data.iloc[i]['text']
            befor_keyowrd, word, after_keyword = mystring.partition(word)
            print (after_keyword)
        
        
        
        
        



Word "staff" found in line 0
 wa horrible u 
Word "staff" found in line 106
 friendly food ha gotten better better 
Word "staff" found in line 288
 know know menu meal need 
Word "staff" found in line 342
 welcome dont ask help get 
Word "staff" found in line 367
 accomodating 
Word "staff" found in line 420
 
Word "staff" found in line 450
 friendly overly efficient 
Word "staff" found in line 520
 nice husband enjoyed meal 
Word "staff" found in line 547
 friendly owner menu keep u coming back 
Word "staff" found in line 609
 nonsense 
Word "staff" found in line 614
 
Word "staff" found in line 647
 actually let enjoy meal youre 
Word "staff" found in line 676
 wa amazing u 
Word "staff" found in line 687
 head past u 
Word "staff" found in line 718
 attentive helpful impressive feat close quarter 
Word "staff" found in line 785
 fabulous bistro fare wonderful jazz brunch great live jazz awesome 
Word "staff" found in line 956
 wa seating guest arrived reservation probably lot walk c

Word "wa" found in line 1157
 always go minnow despite living park 
Word "wa" found in line 1159
ys nice crowd never loud 
Word "wa" found in line 1160
 cooked friend salmon wa completely raw 
Word "wa" found in line 1170
 
Word "wa" found in line 1173
snt u though 
Word "wa" found in line 1182
iting 
Word "wa" found in line 1183
ys good always yummy especially warm pita bread 
Word "wa" found in line 1184
itress great everything menu price food really cheap service get 
Word "wa" found in line 1191
ys someone take empty seat seems customer service ha 
Word "wa" found in line 1194
y overpriced 
Word "wa" found in line 1196
y better 
Word "wa" found in line 1202
y table next u walked table serf much longer 
Word "wa" found in line 1203
 party 7 close next front door eating coat 
Word "wa" found in line 1209
 disappointed restaurant 
Word "wa" found in line 1210
ys made wine selection fairly priced 
Word "wa" found in line 1211
 fresh 
Word "wa" found in line 1214
 place rare gem ny 
Wor

Word "u" found in line 483
 wa except salmon 
Word "u" found in line 484
te cozy 
Word "u" found in line 488
de everything could rush u 
Word "u" found in line 489
ldnt ordering anything else simple asian appetizer drink 
Word "u" found in line 491
st say view nyc beautiful 
Word "u" found in line 492
d like nice light meal asian long good place 
Word "u" found in line 496
e excellent sushi service 
Word "u" found in line 497
ld much better served run group actually customer service 
Word "u" found in line 498
de 
Word "u" found in line 501
sic doe get little loud time made want beautiful wife got 
Word "u" found in line 504
ld go 
Word "u" found in line 505
rant finally decided go dinner 
Word "u" found in line 506
nt people restaurant nice dine 
Word "u" found in line 507
rry friend ordered pad thai chicken 
Word "u" found in line 508
e 
Word "u" found in line 509
k tuk second favorite neighborhood thai place first joya 
Word "u" found in line 510
p hit spot 
Word "u" found in line 5

rant 
Word "u" found in line 2029
e world worth price lamb great 
Word "u" found in line 2032
toftowners get big location 
Word "u" found in line 2033
rned regularly friend 
Word "u" found in line 2036
sly chinese place doe mean big actually mean great 
Word "u" found in line 2037
rant wa loud full romantic 
Word "u" found in line 2038
sband mom girlfriend larger group 
Word "u" found in line 2041
thentic thai restaurant 
Word "u" found in line 2042
ally gave another chance second visit wa even worse 
Word "u" found in line 2045
nday morning 
Word "u" found in line 2047
 ha lot option hope go back try potato pancake 
Word "u" found in line 2048
rday sunday first thing mind getting fried dumpling 
Word "u" found in line 2050
sually order one one split tend offer large portion 
Word "u" found in line 2051
rant wa way 
Word "u" found in line 2052
stomer 
Word "u" found in line 2053
s crab cake 
Word "u" found in line 2054
se 
Word "u" found in line 2055
shing eat 
Word "u" found in line 2

 wa authentic 
Word "food" found in line 1656
 ive ever 
Word "food" found in line 1675
 wa spectacular view 
Word "food" found in line 1678
 freshness 
Word "food" found in line 1682
 
Word "food" found in line 1695
 
Word "food" found in line 1701
 wa good 
Word "food" found in line 1739
 although crowded 
Word "food" found in line 1743
 wa average service wa pathetic wa ambience 
Word "food" found in line 1753
 
Word "food" found in line 1760
 
Word "food" found in line 1772
 drink 
Word "food" found in line 1783
 
Word "food" found in line 1808
 wa good view new york city wa terrific even day like went 
Word "food" found in line 1810
 nothing like menu 
Word "food" found in line 1815
 decent best ambience well matter opinion may consider sweet thing thought wa annoying 
Word "food" found in line 1822
 average midtown better heartland brewery city 
Word "food" found in line 1824
 wa absolutely horrible 
Word "food" found in line 1825
 doesnt cooked high turnover pretty fresh 
Word "

...

 range 
Word "per" found in line 1122
 table 
Word "per" found in line 1129
ience 
Word "per" found in line 1134
 thai cuisine 
Word "per" found in line 1199
ience uncomfortable 
Word "per" found in line 1205
sonal herb garden 
Word "per" found in line 1226
 eye started food wa decent great 
Word "per" found in line 1245
 east side snob 
Word "per" found in line 1261
 person two people 
Word "per" found in line 1267
ience taste bud 
Word "per" found in line 1271
ience 
Word "per" found in line 1318
haps last reviewer wa treated attitude life never anything perfect experience 
Word "per" found in line 1319
ience ive ever 
Word "per" found in line 1321
fect nice weather 
Word "per" found in line 1340
ience lived ny last year 
Word "per" found in line 1353
ience though people walking street 
Word "per" found in line 1416
 sauce wa red 
Word "per" found in line 1437
fect 
Word "per" found in line 1446
ience 
Word "per" found in line 1451
fect looking place close grand central 
Word "per" f

 worth take bite 
Word "priced" found in line 1597
 
Word "priced" found in line 1698
 wine 
Word "priced" found in line 1837
 
Word "priced" found in line 1867
 atmosphere filled beautiful people course food wa surprising good 
Word "priced" found in line 1895
 soggy 
Word "priced" found in line 2002
 
Word "priced" found in line 2261
 meal 
Word "priced" found in line 2356
 
Word "priced" found in line 2382
 
Word "priced" found in line 2389
 noodle 
Word "priced" found in line 2486
 nyc 
Word "priced" found in line 2543
 
Word "priced" found in line 2577
 
Word "priced" found in line 2630
 
Word "priced" found in line 2803
 around 8 dinner 5 lunch delicious filling 
Word "priced" found in line 2877
 tasty 
Word "searching" found in line 592
 get cold weather went thought wa deli 
Word "searching" found in line 606
 really hard restaurant new york could really feel ignored finally found 
Word "searching" found in line 913
 another place 
Word "cold" found in line 592
 weather went th

 restaurant offer dessert beyond complimentary filled chocolate 
Word "turnover" found in line 2872
 
Word "pm" found in line 717
 pretheater menu 1995 tomato soup steak frites wa 
Word "pm" found in line 782
 call 
Word "pm" found in line 1297
 
Word "pm" found in line 1766
 wa packed noisy 
Word "pm" found in line 1894
 
Word "pm" found in line 3041
 restaurant wa practically empty 
Word "1995" found in line 717
 tomato soup steak frites wa 
Word "1995" found in line 1777
 
Word "1995" found in line 1833
 eat 
Word "frites" found in line 717
 wa 
Word "frites" found in line 1438
 came filet wa good u 
Word "frites" found in line 2766
 delicious bit side 
Word "feat" found in line 470
ure seating unlike joint ha bar deserves mention 
Word "feat" found in line 718
 close quarter 
Word "feat" found in line 1216
 good french food area 
Word "feat" found in line 1334
 
Word "feat" found in line 1481
ure nice bar ha great staff 
Word "feat" found in line 1586
ure suan 
Word "feat" found in

 gem casual hip well prepared basic french bistro fare good special warm atmosphere 
Word "village" found in line 1853
 nyc restaurant stay open late 
Word "village" found in line 1920
 sushi restaurant 
Word "village" found in line 2303
 believe food ha slightly quality 
Word "village" found in line 2333
 romantic simple small 
Word "village" found in line 2385
 
Word "village" found in line 2687
 ive never disappointed 
Word "village" found in line 2839
 one place score front 
Word "village" found in line 2957
 
Word "section" found in line 916
 menu called american chinese food 
Word "section" found in line 1629
 
Word "section" found in line 1650
 menu nothing got wa thai 
Word "section" found in line 2970
 menu 
Word "called" found in line 916
 american chinese food 
Word "called" found in line 1281
 morning didnt think would able get 12 wa able get along four guest 
Word "called" found in line 1726
 reputation 
Word "called" found in line 1888
 thank u coming girlfriend enjoyed b

 wa place came recommended 
Word "pastis" found in line 2998
 
Word "general" found in line 1091
 chicken place 
Word "general" found in line 2279
 dining 
Word "general" found in line 2332
 
Word "sharing" found in line 1096
 one entree 
Word "sharing" found in line 1546
 conversation next table trying speak traffic 
Word "sharing" found in line 1914
 table typical dim sum atmosphere sleek chinatown 
Word "sharing" found in line 2185
 main entree cold end meal 
Word "sharing" found in line 2688
 large pizza two special one wa goat cheese blue cheese excellent 
Word "sharing" found in line 2905
 lazy table even family feel comfortable 
Word "waited" found in line 1101
 bar 20 minute seated 
Word "waited" found in line 1687
 well half hour table even though reservation 
Word "waited" found in line 1799
 
Word "waited" found in line 2147
 one hour food 
Word "italy" found in line 1105
 one bite 
Word "italy" found in line 1545
 thing rushed sit dinner table night 
Word "italy" found in l

### Co-occurance Matrix in Python

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
docs = ['this this this book',
        'this cat good',
        'cat good shit']
count_model = CountVectorizer(ngram_range=(1,1)) # default unigram model

X = count_model.fit_transform(docs)
#print(X)
Xc = (X.T * X) # this is co-occurrence matrix in sparse csr format
print(Xc)
Xc.setdiag(0) # sometimes you want to fill same word cooccurence to 0
print(Xc.todense()) # print out matrix in dense format


  (4, 0)	3
  (0, 0)	1
  (3, 1)	1
  (4, 1)	1
  (1, 1)	2
  (2, 1)	2
  (3, 2)	1
  (4, 2)	1
  (1, 2)	2
  (2, 2)	2
  (1, 3)	1
  (2, 3)	1
  (3, 3)	1
  (1, 4)	1
  (2, 4)	1
  (4, 4)	10
  (0, 4)	3
[[0 0 0 0 3]
 [0 0 2 1 1]
 [0 2 0 1 1]
 [0 1 1 0 0]
 [3 1 1 0 0]]


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
samples = ['awesome unicorns are awesome','batman forever and ever','I love batman forever']
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), vocabulary = {'awesome unicorns':0, 'batman forever':1}) 
co_occurrences = bigram_vectorizer.fit_transform(samples)
print ('Printing sparse matrix:', co_occurrences)
print ('Printing dense matrix (cols are vocabulary keys 0-> "awesome unicorns", 1-> "batman forever")', co_occurrences.todense())
sum_occ = np.sum(co_occurrences.todense(),axis=0)
print ('Sum of word-word occurrences:', sum_occ)
print ('Pretty printig of co_occurrences count:', zip(bigram_vectorizer.get_feature_names(),np.array(sum_occ)[0].tolist()))

Printing sparse matrix:   (0, 0)	1
  (1, 1)	1
  (2, 1)	1
Printing dense matrix (cols are vocabulary keys 0-> "awesome unicorns", 1-> "batman forever") [[1 0]
 [0 1]
 [0 1]]
Sum of word-word occurrences: [[1 2]]
Pretty printig of co_occurrences count: <zip object at 0x0000028A9C86E748>


In [15]:
# Check the word is preseint in the sentenec or not
mystring = "I love my country"
word = "country"
if word in mystring: 
    print ('success')

success


In [17]:
# For the position of the word in the sentence
str1In = "I like to code a few things and code a lot"
str2 = "code"
str1 = str1In.split()

indexCount = 0
for word in str1:
    if word == str2:
       print("Your word was found at index point",int(indexCount))
    
    indexCount += 1
    

Your word was found at index point 3
Your word was found at index point 8


In [18]:
mystring =  "hi my name is ryan, and i am new to python and would like to learn more"
keyword = 'name'
befor_keyowrd, keyword, after_keyword = mystring.partition(keyword)
# before_keyword = 'hi my '
# keyword = name
# after_keyword = ' is ryan, and i am new to python and would like to learn more'

In [21]:
befor_keyowrd


'hi my '

In [22]:
text_data

NameError: name 'text_data' is not defined